In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from scipy.stats import chi2, norm, cauchy
from iminuit import Minuit
from iminuit.cost import LeastSquares
# Cambiare directory per spostarsi nella cartella principale
import os
while not os.getcwd().endswith('Laboratorio_SiPM'):
    os.chdir('..')

#importo libreria
import sys
sys.path.append('Config')
from lablib import *


In [ ]:
#definisco le varie funzioni
def gauss(x, mu, sigma, a):
    return a * np.exp(-0.5 * ((x - mu) / sigma) ** 2)

def gauss_cost(x, mu, sigma, a, cost):
    return gauss(x, mu, sigma, a) + cost

def gauss_pol2(x, mu, sigma, a, b, c, cost):
    return gauss(x, mu, sigma, a) + b*x + c * x**2 + cost

def gauss_exp(x, mu, sigma, a, b):
    return gauss(x, mu, sigma, a) + b * np.exp(-x) 

def gauss_pol3 (x, mu, sigma, a, b, c, cost, d):
    return gauss_pol2(x, mu, sigma, a, b, c, cost) + d* x**3

# SODIO

In [ ]:
#prendo i dati
df = pd.read_csv('Dati/Parte 2/BGO/Sodio_55.5V.txt', delim_whitespace= True, skiprows = 65, header=None, encoding= 'ISO-8859-1', names= ['ADC', 'Counts'])
df_bkg = pd.read_csv ('Dati/Parte 2/BGO/Bkg_55.5V.txt', delim_whitespace = True, header = None, skiprows = 65, encoding = 'ISO-8859-1', names = ['ADC', 'Counts'])
fig, ax = plt.subplots(figsize = (30,6))
plt.scatter(df['ADC'][:120], df['Counts'][:120], label = 'Spettro di Na', color = 'blue')
plt.plot(df_bkg['ADC'][:120], df_bkg['Counts'][:120], label = 'Spettro di fondo', color = 'red')
plt.legend()
plt.title('Cristallo BGO Sodio')

## Primo picco

In [ ]:
start_index = 35
end_index = 51
df_primo_picco_sodio = df.iloc[start_index:end_index]
err_primo_picco_sodio = np.array(np.sqrt(df_primo_picco_sodio['Counts'][:]))
#print(err_primo_picco_sodio)


In [ ]:
pars, vals, ers, *other = LS_fit(df_primo_picco_sodio['ADC'], df_primo_picco_sodio['Counts'], err_primo_picco_sodio, gauss, mu = 3170, sigma = 350, a = 7400 )

In [ ]:
for i in range(len(vals)):
    print(f'{pars[i]} = {vals[i]} +/- {ers[i]}')

## Secondo picco

In [ ]:
start_index = 79
end_index = 105
df_secondo_picco_sodio = df.iloc[start_index : end_index]
err_secondo_picco_sodio = np.array(np.sqrt(df_secondo_picco_sodio['Counts']))
plt.plot(df_secondo_picco_sodio['ADC'], df_secondo_picco_sodio['Counts'])
#print(df_secondo_picco_sodio)

In [ ]:
pars, vals, errs, *others = LS_fit(df_secondo_picco_sodio['ADC'], df_secondo_picco_sodio['Counts'], err_secondo_picco_sodio, gauss, mu = 8000, sigma = 350, a = 500)

# COBALTO

In [ ]:
# prendo i dati
df = pd.read_csv('Dati/Parte 2/BGO/Cobalto_55.5V_bins100.txt', delim_whitespace= 'True', skiprows = 65, header=None, encoding= 'ISO-8859-1', names= ['ADC', 'Counts'])
df_bkg = pd.read_csv ('Dati/Parte 2/BGO/Bkg_55.5V.txt', delim_whitespace = 'True', header = None, skiprows = 65, encoding = 'ISO-8859-1', names = ['ADC', 'Counts'])
fig, ax = plt.subplots(figsize = (30,6))
plt.scatter(df['ADC'][:120], df['Counts'][:120], label = 'Spettro di Na', color = 'blue')
plt.plot(df_bkg['ADC'][:120], df_bkg['Counts'][:120], label = 'Spettro di fondo', color = 'red')
plt.legend()
plt.title('Cristallo BGO Cobalto')

In [ ]:
df_filtrato ['ADC'] = df['ADC'][:100] 
df_filtrato['Counts'] = df['Counts'][0:100] - df_bkg['Counts'][0:100]
#print(df_filtrato['Counts'][:])
fig, ax = plt.subplots(figsize = (30,6))
plt.scatter(df_filtrato['ADC'], df_filtrato['Counts'])
start_index = 9
end_index = 25
df_filtrato = df.iloc[start_index:end_index]
df_bkg = df_bkg[start_index:end_index]
err_df_filtrato = np.array(np.sqrt(df_filtrato['Counts']) + np.sqrt(df_bkg['Counts']))


In [ ]:
pars, vals, ers, *other = LS_fit(df_filtrato['ADC'], df_filtrato['Counts'], err_df_filtrato, gauss_cost, mu = 500, sigma = 250, a  = 1000, cost = 70)

In [ ]:
for i in range(len(vals)):
    print(f'{pars[i]} = {vals[i]} +/- {ers[i]}')